<a href="https://colab.research.google.com/github/priyadarshpv/NLP-Driven_Fake_Review_Detection_System/blob/main/NLP_Driven_Fake_Review_Detection_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd

In [3]:
df=pd.read_csv('/content/drive/MyDrive/data sets/fake reviews dataset.csv')

In [4]:
df

,category,rating,label,text_
0,Home_and_Kitchen_5,5.0,CG,"Love this! Well made, sturdy, and very comfor..."
1,Home_and_Kitchen_5,5.0,CG,"love it, a great upgrade from the original. I..."
2,Home_and_Kitchen_5,5.0,CG,This pillow saved my back. I love the look and...
3,Home_and_Kitchen_5,1.0,CG,"Missing information on how to use it, but it i..."
4,Home_and_Kitchen_5,5.0,CG,Very nice set. Good quality. We have had the s...
...,...,...,...,...
40427,Clothing_Shoes_and_Jewelry_5,4.0,OR,I had read some reviews saying that this bra r...
40428,Clothing_Shoes_and_Jewelry_5,5.0,CG,I wasn't sure exactly what it would be. It is ...
40429,Clothing_Shoes_and_Jewelry_5,2.0,OR,"You can wear the hood by itself, wear it with ..."
40430,Clothing_Shoes_and_Jewelry_5,1.0,CG,I liked nothing about this dress. The only rea...


In [5]:
df['label'].value_counts()

,count
label,
CG,20216
OR,20216


In [6]:
df.drop(columns='category',inplace=True)

NLTK preprocessing

In [7]:
import string
punc=string.punctuation


import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
stop_words=stopwords.words('english')
stop_words


from nltk.tokenize import word_tokenize
nltk.download('punkt_tab')


from nltk.stem import PorterStemmer,WordNetLemmatizer
lemmatizer=WordNetLemmatizer()
nltk.download('wordnet')


def preprocess(sentence):
  sen1="".join([i for i in sentence.lower() if i not in punc])
  tokens=[lemmatizer.lemmatize(i,"v") for i in word_tokenize(sen1) if i not in stop_words]
  return " ".join(tokens)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [8]:
x_vector=df['text_'].apply(preprocess)

TF-IDF vectorizer

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer=TfidfVectorizer()
x_vector=vectorizer.fit_transform(df['text_'])

In [10]:
 x_vector.shape

(40432, 41099)

In [11]:
df['label']=df['label'].replace(['CG','OR'],[0,1])

<ipython-input-11-52346a7bc6fa>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['label']=df['label'].replace(['CG','OR'],[0,1])


In [12]:
y=df['label']

In [13]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_vector,y,test_size=0.2,random_state=42)

In [14]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression()
model.fit(x_train,y_train)

LogisticRegression()

In [15]:
y_pred=model.predict(x_test)

In [16]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.91      0.88      0.90      4016
           1       0.89      0.92      0.90      4071

    accuracy                           0.90      8087
   macro avg       0.90      0.90      0.90      8087
weighted avg       0.90      0.90      0.90      8087



In [17]:
def fake_pred(txt):
  s=preprocess(txt)
  l=vectorizer.transform([s])
  y=model.predict(l)[0]
  if y==0:
    print('Computer Generated Review')
  else :
    print('Original Review')

In [18]:
fake_pred('"The wireless Bluetooth headphones offer superior sound quality and a seamless connection. With an impressive battery life of up to 20 hours, they are perfect for long listening sessions. The ergonomic design ensures comfort during prolonged use, and the noise-cancellation feature is effective in eliminating background noise. Whether you are commuting, working, or enjoying music, these headphones provide an enjoyable audio experience. The sleek design adds a modern touch, making them a stylish accessory as well. Overall, a great product for those seeking high-quality audio on the go."')

Original Review


In [19]:
fake_pred("I recently purchased the XYZ Mobile and it has exceeded my expectations in every way. The design is sleek, and the display is absolutely stunning with vibrant colors and crisp details. The 6.5-inch screen is perfect for watching videos, and the touch response is quick and smooth.The camera quality is fantastic—both front and rear cameras produce clear, high-quality photos even in low light. The battery life is impressive, easily lasting a full day with moderate use. The phone also supports fast charging, which is incredibly convenient.The performance is flawless with its latest processor, and it handles multitasking and gaming with ease. I also appreciate the clean user interface with minimal bloatware.Overall, the XYZ Mobile offers incredible value for the price. Highly recommend it for anyone looking for a reliable and feature-packed smartphone.")

Original Review


In [20]:
fake_pred("The iPhone 14 is a top-tier smartphone that combines sleek design with powerful performance. Featuring a Super Retina XDR display, it delivers vibrant colors and sharp visuals.!!wow!! The A15 Bionic chip ensures smooth and responsive performance, whether you're gaming, multitasking, or using demanding apps.")

Original Review


In [21]:
fake_pred('good product')

Computer Generated Review
